In [ ]:
# https://stackoverflow.com/questions/47805170/whats-the-hardware-spec-for-google-colaboratory

In [ ]:
!df -h

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!cat /proc/meminfo

# Как научить компьютер читать?

В этой тетрадке мы обучим свой собственный word2vec. Делать мы это будем на каком-нибудь не очень большом тексте, который вам предстоит выбрать самому. На выбор есть [несколько сказок](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/vector-school) и других [литературных штук](https://github.com/nevmenandr/word2vec-russian-novels/tree/master/books_before) из школьной программы.

In [1]:
# Ссылка на выбранное вами произведение
# Возьмем преступление и наказание
url = 'https://raw.githubusercontent.com/nevmenandr/word2vec-russian-novels/master/books_before/CrimeAndPunishment.txt'

Спарсим текст из файлика.\
**`Requests`** [tutorial](https://realpython.com/python-requests/)

In [3]:
import requests

resp = requests.get(url)
text = resp.text

# Последние 500 символов. Аккуратно! Спойлеры!
print(text[-500:])

смотреть на эти семь лет, как на семь
дней. Он даже и не знал того, что новая жизнь не даром же ему достается, что ее надо еще дорого купить, заплатить за нее великим, будущим подвигом...
Но тут уж начинается новая история, история постепенного обновления человека, история постепенного перерождения его, постепенного перехода из одного мира в другой, знакомства с новою, доселе совершенно неведомою действительностью. Это могло бы составить тему нового рассказа, - но теперешний рассказ наш окончен.


## 1. Предобработка

Теперь нам надо его немного предобработать.  Пусть все слова пишутся с маленькой буквы.

In [4]:
text = text.lower()

Разобьём весь текст на предложения. \
**`re`** [tutorial](https://tproger.ru/translations/regular-expression-python/)

In [5]:
import re
# выкидываем лишние символы!
text = re.sub('\n|\t|\r', ' ', text)

**`nltk`** [tutorial](https://www.guru99.com/nltk-tutorial.html)

In [6]:
import nltk

# nltk.download('all') # если хотим всё и сразу

# нам хватит вот этого:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /home/malleo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/malleo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
from nltk.tokenize import sent_tokenize

sents = sent_tokenize(text)

len(sents)

13702

In [7]:
sents[220]

'действительно, на его платье и даже в волосах кое-где виднелись прилипшие былинки сена.'

Разобьём каждое предложение на отдельные слова.

In [10]:
from nltk.tokenize import word_tokenize

print(word_tokenize(sents[0]), '\n')
sents[0]

['преступление', 'и', 'наказание', 'роман', 'в', 'шести', 'частях', 'с', 'эпилогом', 'часть', 'первая', 'i', 'в', 'начале', 'июля', ',', 'в', 'чрезвычайно', 'жаркое', 'время', ',', 'под', 'вечер', ',', 'один', 'молодой', 'человек', 'вышел', 'из', 'своей', 'каморки', ',', 'которую', 'нанимал', 'от', 'жильцов', 'в', 'с', '-', 'м', 'переулке', ',', 'на', 'улицу', 'и', 'медленно', ',', 'как', 'бы', 'в', 'нерешимости', ',', 'отправился', 'к', 'к', '-', 'ну', 'мосту', '.'] 


'преступление и наказание роман  в шести частях с эпилогом  часть первая  i  в начале июля, в чрезвычайно жаркое время, под вечер, один молодой человек вышел из своей каморки, которую нанимал от жильцов в с - м переулке, на улицу и медленно, как бы в нерешимости, отправился к к - ну мосту.'

In [11]:
from nltk.tokenize import RegexpTokenizer

tokenizer = RegexpTokenizer('\w+')
tokenizer.tokenize(sents[0])

['преступление',
 'и',
 'наказание',
 'роман',
 'в',
 'шести',
 'частях',
 'с',
 'эпилогом',
 'часть',
 'первая',
 'i',
 'в',
 'начале',
 'июля',
 'в',
 'чрезвычайно',
 'жаркое',
 'время',
 'под',
 'вечер',
 'один',
 'молодой',
 'человек',
 'вышел',
 'из',
 'своей',
 'каморки',
 'которую',
 'нанимал',
 'от',
 'жильцов',
 'в',
 'с',
 'м',
 'переулке',
 'на',
 'улицу',
 'и',
 'медленно',
 'как',
 'бы',
 'в',
 'нерешимости',
 'отправился',
 'к',
 'к',
 'ну',
 'мосту']

In [12]:
# разбейте все предложения на токены
sents_tokenize  =  [tokenizer.tokenize(item) for item in sents]
sents_tokenize[:2]

[['преступление',
  'и',
  'наказание',
  'роман',
  'в',
  'шести',
  'частях',
  'с',
  'эпилогом',
  'часть',
  'первая',
  'i',
  'в',
  'начале',
  'июля',
  'в',
  'чрезвычайно',
  'жаркое',
  'время',
  'под',
  'вечер',
  'один',
  'молодой',
  'человек',
  'вышел',
  'из',
  'своей',
  'каморки',
  'которую',
  'нанимал',
  'от',
  'жильцов',
  'в',
  'с',
  'м',
  'переулке',
  'на',
  'улицу',
  'и',
  'медленно',
  'как',
  'бы',
  'в',
  'нерешимости',
  'отправился',
  'к',
  'к',
  'ну',
  'мосту'],
 ['он',
  'благополучно',
  'избегнул',
  'встречи',
  'с',
  'своею',
  'хозяйкой',
  'на',
  'лестнице']]

In [13]:
# Flatten без numpy :)
words = [item for sent in sents_tokenize for item in sent]
words[:10]

# Конструкция выше аналогично этому
# words = []
# for sent in sents_tokenize:
#     for item in sent:
#         if True:
#             words.append(item)

['преступление',
 'и',
 'наказание',
 'роман',
 'в',
 'шести',
 'частях',
 'с',
 'эпилогом',
 'часть']

In [ ]:
len(words) # всего слов

173403

In [ ]:
len(set(words)) # уникальных слов

24925

Можно выбросить все стоп-слова.

In [14]:
from nltk.corpus import stopwords

stopwords_ru = stopwords.words('russian')
stopwords_ru[:10]

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со']

In [ ]:
len(stopwords_ru)

151

In [15]:
# избавьтесь от стоп-слов
sents_tokenize = [[item for item in sent if item not in stopwords_ru]
                        for sent in sents_tokenize]
sents_tokenize[:2]

[['преступление',
  'наказание',
  'роман',
  'шести',
  'частях',
  'эпилогом',
  'часть',
  'первая',
  'i',
  'начале',
  'июля',
  'чрезвычайно',
  'жаркое',
  'время',
  'вечер',
  'молодой',
  'человек',
  'вышел',
  'своей',
  'каморки',
  'которую',
  'нанимал',
  'жильцов',
  'м',
  'переулке',
  'улицу',
  'медленно',
  'нерешимости',
  'отправился',
  'мосту'],
 ['благополучно', 'избегнул', 'встречи', 'своею', 'хозяйкой', 'лестнице']]

Слов в корпусе не очень много. Давайте лемматизируем их.  В этом нам поможет библиотека **pymorphy2.**

**pymorphy2** — это полноценный морфологический анализатор, целиком написанный на Python. Он также умеет ставить слова в нужную форму (спрягать и склонять). [Документация по pymorphy2.](https://pymorphy2.readthedocs.io/en/latest/)

In [ ]:
!pip install pymorphy3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 72.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13707 sha256=47d03e32fcee64585a441f8646f2669806aa29d55f06918e33ad95e904873586
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [16]:
import pymorphy3


morph = pymorphy3.MorphAnalyzer()

text = "Филипп пошёл в Авиньон и пленил пап!"
tokens = tokenizer.tokenize(text)  # regexp tokenizer initialized earlier

" ".join(morph.normal_forms(token)[0] for token in tokens)

'филипп пойти в авиньон и пленить папа'

In [ ]:
p = morph.parse('стали')
p

[Parse(word='стали', tag=OpencorporaTag('VERB,perf,intr plur,past,indc'), normal_form='стать', score=0.975342, methods_stack=((DictionaryAnalyzer(), 'стали', 945, 4),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,gent'), normal_form='сталь', score=0.010958, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 1),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,nomn'), normal_form='сталь', score=0.005479, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 6),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,datv'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 2),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn sing,loct'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 5),)),
 Parse(word='стали', tag=OpencorporaTag('NOUN,inan,femn plur,accs'), normal_form='сталь', score=0.002739, methods_stack=((DictionaryAnalyzer(), 'стали', 13, 9),))]

Зачем нужно возвращать словарь в такой конструкции: `morph.normal_forms(token)[0]`? Ответ ниже.

In [ ]:
morph.normal_forms('стали')

['стать', 'сталь']

Обработаем все слова из датасета.

In [17]:
# лемматизируйте все слова из датасета
sents_tokenize = [[morph.normal_forms(item)[0] for item in sent]
                    for sent in sents_tokenize]

In [18]:
# Flatten без numpy :)
words = [item for sent in  sents_tokenize for item in sent]

In [22]:
len(words) # всего слов

93069

In [23]:
len(set(words)) # уникальных слов

11083

Хватит обработок! Мы тут не анализом текстов занимаемся, а нейросетками. Если хочется больше предобработки, хороший мануал по [ссылке](https://nbviewer.jupyter.org/github/FUlyankin/hse_texts_do/blob/master/sem_1/texts_sem1.ipynb).  Давайте построим словарик с частотностями и перейдём к моделированию.

In [19]:
from collections import Counter

word_dict = Counter(words)
word_dict.most_common()[:20]

[('это', 1449),
 ('всё', 963),
 ('знать', 630),
 ('раскольник', 567),
 ('свой', 549),
 ('один', 548),
 ('сказать', 544),
 ('говорить', 536),
 ('человек', 501),
 ('весь', 442),
 ('стать', 441),
 ('мочь', 441),
 ('который', 430),
 ('сам', 430),
 ('такой', 395),
 ('очень', 387),
 ('какой', 379),
 ('соня', 379),
 ('рука', 369),
 ('петрович', 369)]

In [20]:
for item in words:
  print(item)
  break

преступление


In [26]:
words = word_dict.most_common()
len([item for item in words if item[1] >= 3])  # совсем мало :)

4767

## 2. Моделирование

__Основные параметры:__

* данные должны быть итерируемым объектом
* `vector_size` — размер вектора,
* `window` — размер окна наблюдения,
* `min_count` — мин. частотность слова в корпусе,
* `sg` — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* `sample` — порог для downsampling'a высокочастотных слов,
* `workers` — количество потоков,
* `alpha` — learning rate,
* `epochs` — количество эпох обучения,
* `max_vocab_size` — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.


**`gensim`** [documentation](https://radimrehurek.com/gensim/auto_examples/tutorials/run_word2vec.html#sphx-glr-auto-examples-tutorials-run-word2vec-py)\
[Нововведения](https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4) в **gensim 4.0**


In [28]:
!pip install gensim==4.3.2

  Obtaining dependency information for gensim==4.3.2 from https://files.pythonhosted.org/packages/22/40/7d2cce3ad4ad5d02aa68e253e6ea5f0acc381f02f594e235fe00a274faff/gensim-4.3.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 13.8 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.0
    Uninstalling gensim-4.3.0:
      Successfully uninstalled gensim-4.3.0


In [31]:
type(sents_tokenize), len(sents_tokenize)
# sents_tokenize[0][:5]

(list, 13702)

In [21]:
##%%time
from gensim.models.word2vec import Word2Vec

# vector_size - размерность векторов, которые мы хотим обучить
# window - ширина окна контекста
# min_count - если слово встречается реже, для него не учим модель
model = Word2Vec(vector_size=100, window=2, min_count=3, workers=-1)

# строительство словаря, чтобы обучение шло быстрее
model.build_vocab(sents_tokenize)

# обучение модели (еще так можно модель дообучать)
# первый аргумент - наша выборка, генератор будет вкидывать в модель наши тексты, пока они не кончатся
# второй аргумент - число примеров в выборке
# третий аргумент - количество эпох обучения: сколько раз модель пройдётся по всему корпусу текстов
model.train(sents_tokenize, total_examples=model.corpus_count, epochs=100)

# !NB в ситуации, когда у нас огромный корпус, 100 эпох это слишком много!

(0, 0)

Другой способ инициализировать и тренировать модель.

Такой синтаксис под капотом точно так же строит словарь и вызывает метод `train`

In [33]:
model2 = Word2Vec(sentences=sents_tokenize, vector_size=100, window=2, min_count=3, workers=-1, epochs=100)
print(model2)

Word2Vec<vocab=4767, vector_size=100, alpha=0.025>


In [ ]:
model.corpus_count # число примеров в обучающей выборке

13565

Смотрим, сколько в модели слов.

In [35]:
len(model.wv), len(model2.wv)

(4767, 4767)

In [ ]:
# model.wv.key_to_index

In [36]:
'старуха' in model.wv.key_to_index, 'старуха' in model2.wv.key_to_index

(True, True)

In [37]:
model.wv.get_vecattr("старуха", "count"), model2.wv.get_vecattr("старуха", "count")

(72, 72)

## 3. Свойства модели

In [38]:
# вектор слова
model.wv['старуха'][:10], model2.wv['старуха'][:10]

(array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
         0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ],
       dtype=float32),
 array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
         0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ],
       dtype=float32))

In [39]:
model.wv['старуха']

array([ 0.00158984, -0.00595314, -0.00605843,  0.00848177, -0.00034839,
        0.00706192, -0.00888268, -0.00192542, -0.00676588, -0.0072364 ,
       -0.00636767, -0.00366342,  0.00985412,  0.00071263, -0.00128986,
       -0.00021196,  0.00220028,  0.00328956,  0.00056633, -0.00689849,
       -0.00637836,  0.00246468,  0.0023111 ,  0.00327677,  0.00687588,
        0.0021893 , -0.00301268, -0.00315087,  0.00223726,  0.00897257,
       -0.0030497 , -0.00802446,  0.00619691,  0.00513785, -0.00568808,
       -0.00177443,  0.0082801 ,  0.00772546,  0.00053458, -0.0071188 ,
        0.00630224,  0.00835171,  0.00552652, -0.00818296,  0.00981974,
       -0.00156873,  0.00952912, -0.00775152, -0.00391438,  0.00845072,
        0.00804317, -0.00385863,  0.00980804,  0.0012557 , -0.00509944,
        0.00886316, -0.00389039,  0.00057222, -0.00033299, -0.00965943,
        0.0079309 ,  0.00577724,  0.00411733, -0.00939196,  0.00927838,
       -0.00347454,  0.00825144, -0.0055795 ,  0.00483983, -0.00

In [40]:
len(model.wv['старуха'])

100

In [41]:
# размерность вектора
model.wv['старуха'].shape, model2.wv['старуха'].shape

((100,), (100,))

In [42]:
# похожести слов
model.wv.similarity('тварь', 'право'), model2.wv.similarity('тварь', 'право')

(-0.03793419, -0.03793419)

In [43]:
# самые похожие
model.wv.most_similar('тварь', topn=10)

[('добродушный', 0.35238102078437805),
 ('пренебрегать', 0.32258936762809753),
 ('крикнуть', 0.3088306188583374),
 ('доказательство', 0.2879015803337097),
 ('бред', 0.2792375683784485),
 ('посредством', 0.27730658650398254),
 ('такой', 0.27664443850517273),
 ('ровно', 0.2752732038497925),
 ('пообедать', 0.273645281791687),
 ('навзрыд', 0.2704673409461975)]

In [44]:
# арифметика
model.wv.most_similar(positive=['раскольников','соня'],
                       negative=['тварь'])[:10]

[('нуждаться', 0.3254741132259369),
 ('раздражать', 0.31844109296798706),
 ('выжиг', 0.31684333086013794),
 ('холодеть', 0.30329668521881104),
 ('резать', 0.29883095622062683),
 ('чашка', 0.29052457213401794),
 ('вырываться', 0.2874678075313568),
 ('провинция', 0.2853025794029236),
 ('посматривать', 0.2818695902824402),
 ('несчастие', 0.2812053859233856)]

### KeyedVectors

**TL;DR:** the main difference is that KeyedVectors do not support further training. On the other hand, by shedding the internal data structures necessary for training, KeyedVectors offer a smaller RAM footprint and a simpler interface.

[Source](https://radimrehurek.com/gensim/models/keyedvectors.html#)


По сути, KeyedVectors нужны для экономии места на диске при сохранении модели

In [ ]:
#model.save('aaaaa')

#tmp = Word2Vec.load('aaaaa')
# model.wv.save("word2vec.bbb")

In [45]:
from gensim.models import KeyedVectors

# Store just the words + their trained embeddings.
word_vectors = model.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

vector = wv['раскольников']  # Get numpy vector of a word
vector.shape, vector

((100,),
 array([ 0.0006687 ,  0.00648878,  0.00587001, -0.00017738,  0.0023801 ,
        -0.00651387, -0.00936036, -0.00494762, -0.00538789,  0.00058703,
        -0.00396823, -0.00036959,  0.00763455,  0.00973569,  0.00787588,
         0.00238922, -0.00271333,  0.00243172, -0.00268992, -0.00895014,
         0.00689304, -0.00864854,  0.00378768, -0.00395623, -0.00554306,
        -0.00104358, -0.00656298,  0.00858411, -0.00687793, -0.00896055,
         0.00576626,  0.00446401,  0.00473146,  0.00242694, -0.00372551,
        -0.0051173 ,  0.00177215,  0.00834657, -0.00565228, -0.00238244,
         0.00184836,  0.00672219, -0.0063974 , -0.00452775, -0.00263857,
         0.00133483,  0.00537128,  0.00554937, -0.0087095 ,  0.00727261,
        -0.00250222, -0.00284264, -0.00885856, -0.00577508,  0.00944861,
         0.00308753, -0.00804277,  0.0028074 , -0.00707276, -0.00905789,
         0.00386578,  0.00203697,  0.00125465,  0.00884633,  0.0062607 ,
        -0.00507063,  0.00272783,  0.00254

# Применим полученные выше навыки и решим простую задачу анализа тональности твиттов:

Проделаем весь пайплайн от сырых текстов до получения обученной модели.
Отдельно скачиваем файлы с положительно окрашенными твитами и негативно окрашеннными.
Это реальные данные русскоязычного сегмента твиттера.


In [48]:
# from google.colab import drive
# drive.mount('/content/drive')
import pandas as pd
tweets = pd.read_csv('rusentitweet_full.csv')

In [ ]:
# !wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

In [ ]:
# ! head positive.csv

In [ ]:
# !wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

In [ ]:
# ! head negative.csv

Загрузим и посмотрим на наш датасет:

In [50]:
# tweets.loc[tweets['label'] == 'positive']
positive = tweets.loc[tweets['label'] == 'positive']


negative = tweets.loc[tweets['label'] == 'negative']
df = pd.concat([positive, negative])
df.head()

,Unnamed: 0,text,label,id
8,8,@BTS_twt ты такой красивый 😭😭😭🥺💓,positive,1281551551706865665
11,11,@okdaa @wifeyoonminn @Y_Yoon_ Но ты очень крас...,positive,1236273859671834624
13,13,@Susanoo51489064 ВХВХВХ,positive,1289926968608264193
18,18,@yoursungshine малышка спи сладко ❤,positive,1338933271082770433
23,23,как же я люблю титьки https://t.co/C2M8rMDFyc,positive,1218926151227318272


In [51]:
df.drop(columns=['id'], inplace=True)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.sample(5, random_state=40)

,text,label
12028,"За окном снова дождь. Заебал, мы скоро уплывем...",negative
1771,@chikibammasha Это звучит как идея для видео 🤣🤣🤣,positive
11849,@niemalsnoch @veritas_z ну кто ты и кто залина...,negative
10670,"@Zefirni_chel ГОСПОДИ, ДА!!!",positive
13134,@kpru Ковид его забери и в Аду его задержи,negative


In [ ]:
# import pandas as pd

# загружаем положительные твитты
# positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
# # расставляем метки
# positive['label'] = ['positive'] * len(positive)

# # загружаем отрицательные твитты
# negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
# # расставляем метки
# negative['label'] = ['negative'] * len(negative)

# # соединяем два набора данных
# df = positive.append(negative)
# df.head()

In [52]:
len(df)

5712

Возьмём только небольшой кусок датасета, чтобы долго не ждать выполнения препроцессинга и обучения модели:

In [ ]:
# df = df.sample(frac=0.05).reset_index(drop=True)

Проведем стандартный препроцессинг:

In [ ]:
! pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [53]:
import pymorphy3
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook as tqdm
import re

# pymorphy2 - библиотека методов для морфологического анализа (в том числе лемматизации) русскоязычного текста
m = pymorphy3.MorphAnalyzer()

# убираем все небуквенные символы
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text)
    except:
        return []

In [55]:
#@lru_cache(maxsize=128)
# если вы работаете не колабе, можно заменить pymorphy на mystem и раскомментирвать первую строку про lru_cache
def lemmatize(text, pymorphy=m):
    try:
        return " ".join([pymorphy.parse(w)[0].normal_form for w in text])
    except:
        return " "

In [56]:
def clean_text(text):
    return lemmatize(words_only(text))

In [57]:
# распараллелим процесс на 8 копий, чтобы ускорить,
# и к каждому объекту датасета ( = твиту) применим написанную выше функцию препроцессинга
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))

df['lemmas'] = lemmas
df.head()

/tmp/ipykernel_7038/3057299198.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  lemmas = list(tqdm(p.imap(clean_text, df['text']), total=len(df)))


  0%|          | 0/5712 [00:00<?, ?it/s]

,text,label,lemmas
8,@BTS_twt ты такой красивый 😭😭😭🥺💓,positive,bts_twt ты такой красивый
11,@okdaa @wifeyoonminn @Y_Yoon_ Но ты очень крас...,positive,okdaa wifeyoonminn y_yoon_ но ты очень красиво...
13,@Susanoo51489064 ВХВХВХ,positive,susanoo вхвхвха
18,@yoursungshine малышка спи сладко ❤,positive,yoursungshine малышка спать сладко
23,как же я люблю титьки https://t.co/C2M8rMDFyc,positive,как же я любить титька https://t co/c m rmdfyc


Запишем полученные данные в формате для обучения классификатора:

In [58]:
# переводим данные из датафрейма в списки
X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

# разбиваем на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 3827
total test examples 1885


FastText можно использовать для решения задачи классификации, ей нужно подать данные на вход в особенном формате: текстовый файл, в котором одна строка — один объект выборки, в формате:

```txt
__label__<label_name_1>  text_1
...
__label__<label_name_n>  text_n
```

Записываем train и test выборки в файлы в соответствии с форматом выше:

In [59]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

Обучаем классификатор FastText:

In [60]:
# чтобы установить fasstext, можно склонировать его с репозитория

! git clone https://github.com/facebookresearch/fastText.git
! pip install fastText/.

Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1057/1057), done.
remote: Compressing objects: 100% (189/189), done.
remote: Total 3998 (delta 925), reused 892 (delta 863), pack-reused 2941ects:  25% (1018/3998), 2.94 MiB | 569.00 KiB/sReceiving objects:  27% (1080/3998), 3.66 MiB | 695.00 KiB/sReceiving objects:  32% (1280/3998), 3.66 MiB | 695.00 KiB/sReceiving objects:  37% (1480/3998), 3.66 MiB | 695.00 KiB/sReceiving objects:  38% (1520/3998), 6.36 MiB | 1.21 MiB/sReceiving objects:  42% (1680/3998), 6.36 MiB | 1.21 MiB/sReceiving objects:  47% (1880/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  61% (2439/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  75% (2999/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  85% (3399/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  87% (3490/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  91% (3639/3998), 7.45 MiB | 1.40 MiB/sReceiving objects:  98% (3919/3998), 7.45 MiB | 1.40 MiB/s
Receiving

In [61]:
import fasttext

classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

Read 0M words
Number of words:  11321
Number of labels: 2
Progress: 100.0% words/sec/thread:   39258 lr:  0.000000 avg.loss:  0.557720 ETA:   0h 0m 0s


In [62]:
dir(fasttext)

['BOW',
 'EOS',
 'EOW',
 'FastText',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'absolute_import',
 'cbow',
 'division',
 'load_model',
 'print_function',
 'skipgram',
 'supervised',
 'tokenize',
 'train_supervised',
 'train_unsupervised',
 'unicode_literals']

In [63]:
result

(1885, 0.7480106100795756, 0.7480106100795756)

Смотрим на метрики качества (precision и recall) полученной модели:

In [64]:
print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.7480106100795756
R@1: 0.7480106100795756
Number of examples: 1885


## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на *train* и *test* для задачи классификации (в качестве метки класса будем использовать поле `topic`). В качестве данных для классификации возьмите
    - только заголовки (`title`)
    - только тексты новости (`text`)
    - и то, и другое
3. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель) для классификации текстов по темам. Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. Сравните качество для разных данных из п. 2.



In [25]:
! wget -O lenta-ru-news-part.csv https://www.dropbox.com/s/tce5a8gn68rfw8v/lenta-ru-news-part.csv?dl=0

--2024-03-18 20:10:41--  https://www.dropbox.com/s/tce5a8gn68rfw8v/lenta-ru-news-part.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.18, 2620:100:6026:18::a27d:4612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/tce5a8gn68rfw8v/lenta-ru-news-part.csv [following]
--2024-03-18 20:10:42--  https://www.dropbox.com/s/raw/tce5a8gn68rfw8v/lenta-ru-news-part.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc8404ec93c0a7a14b01804716c5.dl.dropboxusercontent.com/cd/0/inline/CPWwUC-RF8p51GpJ1ueipbJTY4kg-AUN0EzMZuBgfKPVhSFWUflkqmG1T_LV0-cWDLm1mNkRBLTP3U_sZ7yqf4w5xhQM0yD3tB-wWSKc_nXbsrR2130JotcXPgYq8l8q1YHW94USb0b_bTjmLTQG48Pc/file# [following]
--2024-03-18 20:10:42--  https://uc8404ec93c0a7a14b01804716c5.dl.dropboxusercontent.com/cd/0/inline/CPWwUC-RF8p51GpJ1ueipbJTY4kg-AUN0EzMZuBgfKPVhSFWUflkqmG1T_LV0-cW

In [23]:
import pandas as pd

In [24]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xd1 in position 0: unexpected end of data

In [68]:
lenta.shape

(258273, 3)

In [69]:
lenta.topic.value_counts() # наш таргет

topic
Экономика          79528
Спорт              64413
Культура           53797
Наука и техника    53136
Бизнес              7399
Name: count, dtype: int64